## TODO:

- Need another metric as well as pct ID to take into account gaps:
    - gaps in domain alignment 

In [1]:
import pandas as pd
from Bio import pairwise2
from Bio import Align

from data_loading import load_valid_isoform_clones

In [2]:
doms = pd.read_excel('../../data/TF_reg_domain_2019-08-13.xlsx',
                   sheet_name='Sheet1')
doms = (doms.loc[:, ['symbol', 'Domain type', 'Domain sequence', 'aa position']]
        .rename(columns={'symbol': 'hgnc_gene_symbol',
                         'Domain type': 'type',
                         'Domain sequence': 'aa_seq',
                         'aa position': 'pos'}))
doms = doms.dropna()
doms['aa_seq'] = doms['aa_seq'].str.replace(' ', '')

In [3]:
doms['type'].value_counts()

Activation                   177
Repression                    84
Predicted repression          81
DNA binding inhibition         5
Activation/Repression          1
Repression (dimerization)      1
Regulatory domain              1
Name: type, dtype: int64

In [4]:
iso = load_valid_isoform_clones()

In [86]:
doms['hgnc_gene_symbol'].isin(iso['gene']).value_counts()

True    350
Name: hgnc_gene_symbol, dtype: int64

In [5]:
# What should the output look like?

# Two tables: 
#   1. remapped domains to single reference isoform which is best match / longest isoform, containing start / end
#   2. one row for each domain and each alternative isoform with how much of domain is in there

In [22]:
# half perfect match, half not
def num_aa_id(algn):
    a = [algn.target[i] for start, stop in algn.aligned[0] for i in range(start, stop)]
    b = [algn.query[i] for start, stop in algn.aligned[1] for i in range(start, stop)]
    if len(a) != len(b):
        raise UserWarning('Something went wrong')
    return sum(l_a == l_b for l_a, l_b in zip(a, b))


aligner = Align.PairwiseAligner()
aligner.mode = 'local'
aligner.open_gap_score = -1
aligner.extend_gap_score = -1
aligner.target_end_gap_score = 0.0
aligner.query_end_gap_score = 0.0


remapped_domains = []
domain_aligments = {}
for i, domain in doms.iterrows():
    isoforms = iso.loc[iso['gene'] == domain['hgnc_gene_symbol'], :]
    is_perfect_match = isoforms['aa_seq'].str.contains(domain['aa_seq'])
    if is_perfect_match.any():
        ref_iso = isoforms.loc[is_perfect_match, :].iloc[0, :]
        start = ref_iso['aa_seq'].index(domain['aa_seq']) + 1
        remapped_domains.append((ref_iso['gene'],
                                 ref_iso['clone_acc'],
                                 start,
                                 start + (len(domain['aa_seq']) - 1),
                                 100,
                                 0))
    else:
        # align every clone, take the best
        alignments = [(iso['clone_acc'], aligner.align(iso['aa_seq'], domain['aa_seq'])[0])
                      for __i, iso in isoforms.iterrows()]
        ref_clone_acc, best_alignment = sorted(alignments, key=lambda x: x[1].score)[-1]
        pct_id = 100 * (num_aa_id(best_alignment) / len(domain['aa_seq']))
        n_gaps = best_alignment.__str__().split()[0].count('-')
        domain_aligments[ref_clone_acc]  = best_alignment
        start = best_alignment.aligned[0][0][0]
        stop = best_alignment.aligned[0][-1][1]
        remapped_domains.append((domain['hgnc_gene_symbol'],
                                 ref_clone_acc,
                                 start,
                                 stop,
                                 pct_id,
                                 n_gaps))
        
remapped_domains = pd.DataFrame(remapped_domains, columns=['gene', 
                                                           'clone_acc', 
                                                           'start', 
                                                           'end', 
                                                           'pct_match',
                                                           'n_gaps_in_iso_align'])

In [33]:
remapped_domains.sort_values('pct_match').head(42)

,gene,clone_acc,start,end,pct_match,n_gaps_in_iso_align
11,NFAT5,NFAT5|1/2|02F01,4,164,13.172542,4
117,MAZ,MAZ|2/3|01G05,0,73,14.112903,3
59,ZNF772,ZNF772|4/4|07E07,14,25,14.285714,1
38,ZBTB47,ZBTB47|2/6|10C05,121,223,14.545455,1
179,ZNF148,ZNF148|2/2|09H01,13,108,17.699115,2
41,ZIM2,ZIM2|1/1|05E03,5,50,19.298246,2
126,NFATC1,NFATC1|2/2|09D10,267,506,21.161826,3
134,PAX4,PAX4|2/2|11F07,20,83,22.077922,2
180,ZNF148,ZNF148|2/2|09H01,1,40,22.448980,2
6,HEY2,HEY2|2/2|05C07,0,16,25.806452,0


In [26]:
remapped_domains.sort_values('n_gaps_in_iso_align', ascending=False).head(20)

,gene,clone_acc,start,end,pct_match,n_gaps_in_iso_align
118,MEF2C,MEF2C|1/2|01F04,86,441,91.472868,32
17,POU2F2,POU2F2|4/4|11A08,98,190,85.185185,16
8,MEF2A,MEF2A|1/2|12D08,85,499,98.104265,8
35,TEAD1,TEAD1|5/5|11E10,134,357,65.236052,6
11,NFAT5,NFAT5|1/2|02F01,4,164,13.172542,4
117,MAZ,MAZ|2/3|01G05,0,73,14.112903,3
294,NFIX,NFIX|4/4|08D05,266,353,52.808989,3
126,NFATC1,NFATC1|2/2|09D10,267,506,21.161826,3
134,PAX4,PAX4|2/2|11F07,20,83,22.077922,2
180,ZNF148,ZNF148|2/2|09H01,1,40,22.448980,2


In [31]:
((remapped_domains['pct_match'] == 100).sum(), remapped_domains.shape[0])

(309, 350)

In [32]:
((remapped_domains['pct_match'] > 95).sum(), remapped_domains.shape[0])

(322, 350)

In [7]:
# check the worst alignments to see if anything went wrong...
doms.loc[doms['hgnc_gene_symbol'] == 'ZBTB47', :]

,hgnc_gene_symbol,type,aa_seq,pos
46,ZBTB47,Predicted repression,LNEQRLFQPDLCDVDLVLVPQRSVFPAHKGVLAAYSQFFHSLFTQN...,4-113 (Q9UFB7-1)


In [8]:
iso.loc[iso['gene'] == 'ZBTB47', :]

,gene,clone_acc,aa_seq,num_aa
548,ZBTB47,ZBTB47|2/6|10C05,MNVTHSRMQICDQCGKRFLLESELLLHRQTDCERNIQCVTCGKAFK...,293


In [20]:
domain_aligments['MEF2C|1/2|01F04'].__str__().split()[0].count('-')

32

## Checking the worst alignments to see if anything went wrong...

1. NFAT5 -- Looks sensible, isoform we have is 242 aa vs 1,455 in uniprot
2. MAZ -- another one where we just have short isoforms
3. ZNF772 -- and another one, with just 38 residues....
4. 

In [48]:
# check length of original domain against length of remapped domain..
old_length = doms.groupby('hgnc_gene_symbol').sum()['aa_seq'].str.len()
new_length = ((remapped_domains['end'] - remapped_domains['start']) + 1).groupby(remapped_domains['gene']).sum()
diff = new_length - old_length
diff.name = 'n_aa_difference'
diff = diff.to_frame()
diff['abs_n_aa_diff'] = diff['n_aa_difference'].abs()
diff['pct_match'] = diff.index.map(remapped_domains.groupby('gene').min()['pct_match'])
diff.sort_values('abs_n_aa_diff', ascending=False).head(20)

,n_aa_difference,abs_n_aa_diff,pct_match
gene,,,
NFAT5,-378,378,13.172542
ZNF148,-244,244,17.699115
MAZ,-174,174,14.112903
ETV4,-112,112,34.682081
PAX4,-70,70,22.077922
HEY2,-45,45,25.806452
CREM,-39,39,61.417323
ZNF85,-32,32,57.142857
MEF2C,-31,31,91.472868


In [63]:
# check in cases with multiple domains per gene, that all domains mapped to the same isform
remapped_domains.loc[(remapped_domains['gene']
                                      .map(remapped_domains.loc[(remapped_domains['pct_match'] > 95), :]
                                                           .groupby('gene')
                                                           ['clone_acc']
                                                           .nunique()) > 1) &
                     (remapped_domains['pct_match'] > 95),
                     :]

,gene,clone_acc,start,end,pct_match,n_gaps_in_iso_align
121,MYF6,MYF6|1/2|03G09,1,92,100.000000,0
122,MYF6,MYF6|2/2|03F09,145,242,100.000000,0
160,THRA,THRA|1/3|09H07,1,52,100.000000,0
161,THRA,THRA|2/3|09H09,376,410,100.000000,0
162,THRB,THRB|2/2|07E08,1,94,100.000000,0
163,THRB,THRB|1/2|07E05,468,476,100.000000,0
223,MEIS2,MEIS2|2/4|12A07,340,470,100.000000,0
224,MEIS2,MEIS2|1/4|08H09,150,193,100.000000,0
238,PRRX1,PRRX1|2/3|09F06,217,245,100.000000,0
239,PRRX1,PRRX1|2/3|09F06,200,216,100.000000,0


In [64]:
doms.loc[doms['hgnc_gene_symbol'] == 'MYF6', :]

,hgnc_gene_symbol,type,aa_seq,pos
148,MYF6,Activation,MMMDLFETGSYFFYLDGENVTLQPLEVAEGSPLYPGSDGTLSPCQD...,1-92(P23409-1)
149,MYF6,Activation,QDLLHRLDQQEKMQELGVDPFSYRPKQENLEGADFLRTCSSQWPSV...,145-242 (P23409-1)


In [79]:
dom_seq = doms.loc[doms['hgnc_gene_symbol'] == 'MYF6', 'aa_seq'].values[1]
iso_a_seq = iso.loc[iso['clone_acc'] == 'MYF6|1/2|03G09', 'aa_seq'].values[0]
iso_b_seq = iso.loc[iso['clone_acc'] == 'MYF6|2/2|03F09', 'aa_seq'].values[0]
(dom_seq in iso_a_seq, dom_seq in iso_b_seq)

(False, True)

In [75]:
dom_seq

'QDLLHRLDQQEKMQELGVDPFSYRPKQENLEGADFLRTCSSQWPSVSDHSRGLVITAKEGGASIDSSASSSLRCLSSIVDSISSEERKLPCVEEVVEK'

In [76]:
iso_a_seq

'MMMDLFETGSYFFYLDGENVTLQPLEVAEGSPLYPGSDGTLSPCQDQMPPEAGSDSSGEEHVLAPPGLQPPHCPGQCLIWACKTCKRKSAPTDRRKAATLRERRRLKKINEAFEALKRRTVANPNQRLPKVEILRSAISYIERLQDLLHRLDQQEKMQELGVDPFSYRPKQENLEGADFLRTCSSQWPSVSDHSRGLVITAKEGKVKGLWAAPEKIREVDRMLGPRGLEARAGTRPAKRALFARRDQAFPRCQSGLARGAELLRVPSLGAMFVCCFFAQEEQVLIRQPRVAFDAFLPSWTVFPRRNANSPAWRKWWR'

In [80]:
iso_b_seq

'MMMDLFETGSYFFYLDGENVTLQPLEVAEGSPLYPGSDGTLSPCQDQMPPEAGSDSSGEEHVLAPPGLQPPHCPGQCLIWACKTCKRKSAPTDRRKAATLRERRRLKKINEAFEALKRRTVANPNQRLPKVEILRSAISYIERLQDLLHRLDQQEKMQELGVDPFSYRPKQENLEGADFLRTCSSQWPSVSDHSRGLVITAKEGGASIDSSASSSLRCLSSIVDSISSEERKLPCVEEVVEK'

In [83]:
(len(iso_a_seq), len(iso_b_seq))

(317, 242)

In the case of MYF6, the second isoform matches the uniprot sequence, therefore both domains should be matched to that isoform.